In [1]:
#Before using spark, need to install and set up spark, Hadoop and Jdk locally on your laptop


from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("etl-apparel") \
    .config("spark.master", "local") \
    .config("spark.jars", "F:/UniPrj/Big Data Project/postgresql-42.2.16.jar") \
    .getOrCreate()

#Load Amazon Data into Spark DataFrame
#1 load product segment - Apparel Segment --> download dataset tsv file
file_path = "F:/UniPrj/Big Data Project/Datasets/amazon_reviews_us_Apparel_v1_00.tsv"
df = spark.read.option("encoding", "UTF-8").csv(file_path, sep="\t", header=True, inferSchema=True)

# Show the first few rows of the DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [2]:
#Create DataFrame - Perform Preliminary Cleaning
#1 Check the schema, print row & column count
df.printSchema()
print((df.count(), len(df.columns)))

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)

(5906333, 15)


In [3]:
#2 Drop the round 1 columns
columns_to_drop = ['marketplace', 'product_parent', 'vine', 'review_headline', 'review_body', 'review_date']
df_dropped = df.drop(*columns_to_drop)
df_dropped.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|verified_purchase|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+-----------------+
|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|Easy Tool Stainle...|         Apparel|          4|            0|          0|                Y|
|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|V28 Women Cowl Ne...|         Apparel|          5|            1|          2|                Y|
|   12608825| RWQEDYAX373I1|B01I497BGY|James Fiallo Men'...|         Apparel|          5|            0|          0|                Y|
|   25482800|R231YI7R4GPF6J|B01HDXFZK6|Belfry Gangster 1...|         Apparel|          5|            0|          0|                Y|
|    9310286|R3KO3W45DD0L1K|B01G6MBEBY|JAEDEN Women's Be...|  

In [4]:
#3 Filter Step 1:
df_filtered = df_dropped.filter(df_dropped.verified_purchase == 'Y')
df_filtered.show
print((df_filtered.count(), len(df_filtered.columns)))

(5312774, 9)


In [5]:
#4 drop filtered verified_purchase column
columns_to_drop = ['verified_purchase']
df_dropped_2 = df_filtered.drop(*columns_to_drop)
df_dropped_2.show()

+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+
|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|Easy Tool Stainle...|         Apparel|          4|            0|          0|
|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|V28 Women Cowl Ne...|         Apparel|          5|            1|          2|
|   12608825| RWQEDYAX373I1|B01I497BGY|James Fiallo Men'...|         Apparel|          5|            0|          0|
|   25482800|R231YI7R4GPF6J|B01HDXFZK6|Belfry Gangster 1...|         Apparel|          5|            0|          0|
|    9310286|R3KO3W45DD0L1K|B01G6MBEBY|JAEDEN Women's Be...|         Apparel|          5|            0|          0|
|   26631939|R1C4QH63NFL5NJ|B01FWRXN0Y|Levi's Boys' 514 ...|         App

In [6]:
# Create Analysis-Specific DFs/Tables
# Perform Analysis-Specific Transforms
# Segmentation Analysis DF

# 1 Create Segmentation DF by Droppig Addtional Columns
segmentation_cols_drop = ['review_id', 'product_id', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
segmentation_dropped_df = df_dropped_2.drop(*segmentation_cols_drop)
segmentation_dropped_df.show()

+-----------+----------------+
|customer_id|product_category|
+-----------+----------------+
|   32158956|         Apparel|
|    2714559|         Apparel|
|   12608825|         Apparel|
|   25482800|         Apparel|
|    9310286|         Apparel|
|   26631939|         Apparel|
|   48785098|         Apparel|
|   39548589|         Apparel|
|   29355866|         Apparel|
|   27477484|         Apparel|
|   17685865|         Apparel|
|   19787539|         Apparel|
|   44196725|         Apparel|
|   45510794|         Apparel|
|   42025740|         Apparel|
|   27097189|         Apparel|
|   13808904|         Apparel|
|   30225894|         Apparel|
|   45223824|         Apparel|
|   22300783|         Apparel|
+-----------+----------------+
only showing top 20 rows



In [7]:
#Note: must change Category Label name in withColumnRenamed('count(product_category)', 'name')
#2 Segmentation GroupBy
#2a GroupBy customer_id
#2b Count product_category and rename count columns as Segment Name
segment_df = segmentation_dropped_df.groupby("customer_id")\
.agg({'product_category':'count'}).withColumnRenamed('count(product_category)', 'apparel')
#2c Check results
segment_df.show()

+-----------+-------+
|customer_id|apparel|
+-----------+-------+
|   13780696|      1|
|   43380682|      1|
|   27446622|      1|
|     120706|      3|
|   15915947|      2|
|   37520920|      1|
|   14309190|      8|
|   22316505|      1|
|   43949054|      2|
|   21102747|      1|
|   19021463|      1|
|   16075459|      7|
|   51475782|      2|
|   35471383|      3|
|   34788104|      2|
|    1169723|      1|
|   40659946|      1|
|    1319956|      1|
|     193166|      2|
|   44514239|      1|
+-----------+-------+
only showing top 20 rows



In [8]:
#3 Check segment_df Schema and Row Count
segment_df.printSchema()
print(segment_df.count())

root
 |-- customer_id: integer (nullable = true)
 |-- apparel: long (nullable = false)

2958291


In [9]:
#Note: Column Name in df.sort('name' ...) must align with Column name from step 2
#4 Filter for Top n Results
#4a Declare number of rows to filter by (100,000)
row_count = 100000
#4a Sort by Segment Desc and limit to row_count
filtered_segment_df = segment_df.sort('apparel', ascending=False).limit(row_count)
#4b Check Results
filtered_segment_df.show()
print(filtered_segment_df.count())

+-----------+-------+
|customer_id|apparel|
+-----------+-------+
|   37474421|    258|
|   45825821|    249|
|   15006109|    223|
|   10558864|    192|
|    6724212|    175|
|   22985631|    165|
|   45187592|    162|
|   14230771|    154|
|   19672530|    152|
|   25388273|    148|
|   37965911|    146|
|   35178127|    143|
|   38166052|    143|
|    7809052|    140|
|   40333751|    140|
|   17580988|    133|
|   42995420|    131|
|    6534413|    127|
|    2174967|    124|
|   43310494|    124|
+-----------+-------+
only showing top 20 rows

100000


In [10]:
#Segmentation ETL Complete - Add Database Export Code

# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [11]:
#Note: table name in table='name_segment' must align with table name in Postgres

# Write segment table to Postgres/RDS
# 1 min 5 secs
filtered_segment_df.write.jdbc(url=jdbc_url, table='apparel_segment', mode=mode, properties=config)
     

In [12]:
# Run Queries in Postgres to Confirm Segment ETL
# Check Row Count of Segment Table - SELECT COUNT(*) FROM apparel_segment;
# Check 10 Rows of Segment Table - SELECT * FROM apparel_segment LIMIT(10);
# Upon Confirmation of Above Checks Run This Cell
print('Segment ETL Successful')

Segment ETL Successful


In [13]:
# Apriori Analysis DF
# Enter Apriori Transfomations Below - Use df_dropped_2 as Start Point

# 1 Create Apriori DF by First Dropping Addtional Columns
apriori_cols_drop = ['product_category', 'product_title', 'star_rating', 'helpful_votes', 'total_votes']
apriori_dropped_df = df_dropped_2.drop(*apriori_cols_drop)
apriori_dropped_df.show()

+-----------+--------------+----------+
|customer_id|     review_id|product_id|
+-----------+--------------+----------+
|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|
|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|
|   12608825| RWQEDYAX373I1|B01I497BGY|
|   25482800|R231YI7R4GPF6J|B01HDXFZK6|
|    9310286|R3KO3W45DD0L1K|B01G6MBEBY|
|   26631939|R1C4QH63NFL5NJ|B01FWRXN0Y|
|   48785098|R2GP65O1U9N7BP|B01EXNH1HE|
|   39548589|R3O29CT5MQQ3XQ|B01E7OL09O|
|   29355866|R1ZECD2AA8QFF6|B01DXHX81O|
|   27477484|R2S79GCF6J89OA|B01DDULIJK|
|   17685865|R1CBCUGNP37MPK|B01BOKOL4A|
|   19787539|R3NU4WUR5ZNV1V|B01B3Q4Q0O|
|   44196725|R32EPCJ3XF8LGE|B01ADDSL9U|
|   45510794|R1XIBC6WQBW31M|B019P1X5XI|
|   42025740|R1P11MVDZ65LMH|B019MDXIXG|
|   27097189|R1OJA3DJL0VDDK|B019438FEG|
|   13808904|R1THWA5YRJLOOF|B0178HGNIA|
|   30225894| R738LCNRSJVXP|B016VIU0QI|
|   45223824|R1N3Z13931J3O9|B016PUU3VO|
|   22300783|R1LBNTP7E8N89Z|B016AQNDM4|
+-----------+--------------+----------+
only showing top 20 rows



In [14]:
#Prelim Apriori ETL Complete - Add Database Export Code
# Configure settings for RDS
mode = "append"
jdbc_url = 'jdbc:postgresql://localhost:5432/AmazonProducts'
config = {"user":'postgres', 
          "password": 'root', 
          "driver":"org.postgresql.Driver"}

In [15]:
#Note: table name in table='segment_apriori' must align with table name in Postgres

# Write segment_apriori table to RDS
# 13 mins 15 secs
apriori_dropped_df.write.jdbc(url=jdbc_url, table='apparel_apriori', mode=mode, properties=config)
     

In [16]:
# Run Queries in Postgres to Confirm Apriori ETL
# Check Row Count of Apriori Table - SELECT COUNT(*) FROM apparel_apriori;
# Check 10 Rows of Apriori Table - SELECT * FROM apparel_apriori LIMIT(10);

# Upon Confirmation of Above Checks Run This Cell
print('Apriori ETL Successful')

Apriori ETL Successful
